In [1]:
from collections import defaultdict
from copy import deepcopy
from pprint import pprint
from tqdm import tqdm

In [155]:
for p in primes(4, 104): 
    q = p
    F.<a> = GF(q)
    d = 3

    R.<x> = PolynomialRing(F)
    p = F.characteristic()
    r = F.degree()

    # PGL = [(x + b)/(c*x + d) for b in F for c in F for d in F if d - b*c != 0] + [1 / (c*x + d) for c in F for d in F if c != 0]

    def check(s, t):
        poly = x^4 - 2*x^3 - s*x^2 - 2*t*x + t
        return not any(root[0]^3 != -t for root in poly.roots())

    # deg3ratls = set(f / (x^3 + g) for f in R.polynomials(max_degree=2) for g in R.polynomials(max_degree=2) if gcd(f, x^3 + g) == 1)

    deg3ratls = set((x^3 + s*x + t) / (x * (x - 1)) for s in F for t in F if t != 0 and 1 + s + t != 0 and check(s, t))

    print(q, ram4Classes())

5 {1: 0, 2: 2, 3: 2, 4: 0, 5: 2}
7 {1: 1, 2: 3, 3: 4, 4: 0, 5: 4}
11 {1: 1, 2: 5, 3: 4, 4: 3, 5: 5}
13 {1: 2, 2: 6, 3: 6, 4: 4, 5: 6}
17 {1: 2, 2: 8, 3: 6, 4: 6, 5: 8}
19 {1: 3, 2: 10, 3: 8, 4: 6, 5: 9}
23 {1: 3, 2: 11, 3: 8, 4: 9, 5: 11}
29 {1: 4, 2: 14, 3: 10, 4: 12, 5: 14}
31 {1: 5, 2: 16, 3: 12, 4: 12, 5: 15}
37 {1: 6, 2: 18, 3: 14, 4: 16, 5: 18}
41 {1: 6, 2: 20, 3: 14, 4: 18, 5: 20}
43 {1: 7, 2: 21, 3: 16, 4: 18, 5: 22}
47 {1: 7, 2: 23, 3: 16, 4: 21, 5: 23}
53 {1: 8, 2: 26, 3: 18, 4: 24, 5: 26}
59 {1: 9, 2: 29, 3: 20, 4: 27, 5: 29}
61 {1: 11, 2: 30, 3: 22, 4: 27, 5: 30}
67 {1: 11, 2: 33, 3: 24, 4: 30, 5: 34}
71 {1: 11, 2: 35, 3: 24, 4: 33, 5: 35}
73 {1: 12, 2: 36, 3: 26, 4: 34, 5: 36}
79 {1: 13, 2: 40, 3: 28, 4: 36, 5: 39}
83 {1: 13, 2: 41, 3: 28, 4: 39, 5: 41}
89 {1: 14, 2: 44, 3: 30, 4: 42, 5: 44}
97 {1: 16, 2: 48, 3: 34, 4: 46, 5: 48}
101 {1: 16, 2: 50, 3: 34, 4: 48, 5: 50}
103 {1: 17, 2: 51, 3: 36, 4: 48, 5: 52}


In [3]:
def numRatlEquivClasses(p, q, n): 
    def A(q, n): 
        a = sum([euler_phi(d) * (q^(2*n/d-2) + (d-1)*(q^(2*n/d)-1)/(q+1)) for d in divisors(gcd(q-1, n)) if d > 1])
        b = sum([euler_phi(d) * (q^(2*floor(n/d)+1) + 1)/(q+1) for d in divisors(q-1) if (d not in divisors(n))])
        return a + b 

    def B(q, n):
        a = sum([euler_phi(d) * (q^(2*n/d-2) + (q+1) * (q^(2*n/d) - (-1)^(n/d))/(q^2 + 1)) for d in divisors(gcd((q+1), n)) if d % 2 == 0])
        b = sum([euler_phi(d) * q^(2*n/d-2) for d in divisors(gcd((q+1), n)) if (d % 2 == 1 and d > 1)])
        c = 1/((q+1)*(q^2+1)) * sum([euler_phi(d) * ((1+(-1)^(n//d))/2 * (1+q)^2 + q*(q^(2 * n//d + 2) -1)) for d in divisors(q+1) if d not in divisors(n)])
        d = 1/(q^2 + 1) * sum([euler_phi(d) * ((-1)^(n//d) * (1+q) + q^(2 * (n//d) + 1) * (q-1)) for d in divisors(q+1) if d in divisors(q+1) if d not in divisors(n)])
        # print("B terms are", a, b, c, d)
        return a + b + d

    def C(q, n):
        if n % p == 0:
            return q^(2*n/p-1) 
        elif n == 1: 
            return 1 
        elif n % p == 1: 
            return q^(2*(n-1)/p-1) * (q-1) 
        else: 
            return 0

    # print("Each components is", A(q, n), B(q, n), C(q, n))
    
    return (q^(2*n-3))/(q^2-1) + 1/(2*(q-1)) * A(q, n) + 1/(2*(q+1)) * B(q, n) + 1/q * C(q,n)

In [13]:
print(numRatlEquivClasses(p, q, d))

34


In [44]:
# very slow code; pls scratch this later on but I needed some data for very small case 
PGL = [(x + b)/(c*x + d) for b in F for c in F for d in F if d - b*c != 0] + [1 / (c*x + d) for c in F for d in F if c != 0]

def valueFreq(f): 
    """given a rational function f, compute the value frequency of it"""
    frequencies = defaultdict(int)
    for a in F:
        try: 
            value = f(a)
        except ZeroDivisionError: 
            value = "infinity" 
        frequencies[value] += 1
    return sorted(frequencies.values()) 

def ratlEquiv(f, g):
    """returns whether or not f is PGL-equiv to g""" 
    if valueFreq(f) != valueFreq(g): 
        return (False,) 
    for alpha in PGL: 
        for beta in PGL: 
            try:
                if alpha(f(beta)) == g: 
                    return (True, alpha, beta)
            except ZeroDivisionError: 
                continue
    return (False,)

In [ ]:
def reallyslowRatlEquivClasses(print_progress=False): 
    classes = defaultdict(set)
    total = len(deg3ratls)
    candidates = deepcopy(deg3ratls)
    if print_progress:
        print(f"{total - len(candidates)} out of {total} done", end="\r")
    while candidates:
        f = candidates.pop()
        classes[f].add(f)
        for alpha in PGL: 
            for beta in PGL: 
                try: 
                    g = alpha(f(beta))
                    if g in candidates:
                        classes[f].add(g)
                        candidates.discard(g)
                except ZeroDivisionError: 
                    continue
        # classes.add(f)
        if print_progress:
            print(f"{total - len(candidates)} out of {total} done", end="\r")
    if print_progress:
        print()
    return classes

In [22]:
classes = reallyslowRatlEquivClasses(print_progress=True)
print(len(classes))
pprint(classes.keys())

# value_freqs = []
# for f in tqdm(classes):
#     value_freqs.append((f, valueFreq(f)))
# pprint(value_freqs)

100842 out of 100842 done
16
dict_keys([(5*x^2 + 6*x + 5)/(x^3 + 3*x + 6), (6*x^2 + 5*x + 2)/(x^3 + x^2 + 4*x), (5*x^2 + 6*x + 4)/(x^3 + x^2 + 4*x), (5*x^2 + 6*x + 4)/(x^3 + 4*x^2 + x + 2), (6*x^2 + 5*x + 2)/(x^3 + 4*x^2 + x + 2), (5*x^2 + x)/(x^3 + 4*x^2 + 3*x + 5), (2*x^2 + 5*x + 2)/(x^3 + 4*x^2 + 3*x + 1), (3*x^2 + 2*x + 4)/(x^3 + 5*x^2 + 5*x), (6*x^2 + 4*x + 2)/(x^3 + 5*x^2 + x), (5*x^2 + 6*x + 1)/(x^3 + 5*x^2 + 6*x + 1), (4*x + 3)/(x^3 + 6*x^2 + 6*x + 4), (4*x^2 + 6)/(x^3 + 6*x^2 + 6*x + 4), (4*x^2 + 6*x + 1)/(x^3 + x^2 + 5*x + 6), (6*x^2 + 4*x)/(x^3 + 4), (3*x^2 + 3*x + 6)/(x^3 + 3*x^2 + 4*x + 1), (5*x^2 + 5*x + 1)/(x^3 + x^2 + 4*x + 4)])


In [31]:
def getOrbit(f):
    s = f.numerator()[1]
    t = f.numerator()[0]

    preimages = defaultdict(set)
    for a in F:
        if a != 0 and a != 1:
            preimages[f(a)].add(a)
    
    pairs = { (s, t) }
    for preimage in preimages.values():
        if len(preimage) == 3:
            u, v, w = preimage
            s_prime = (u^2*(-1 + v)*v*(u - w) - u*(-1 + v)*v^2*(u - w) + u*(u - v)*v^2*(-1 + w) + (1 - v)*v^2*(u - w)*w + (-1 + v)*v^2*(u - w)*w - u^2*(u - v)*(-1 + w)*w + v^2*(-u + v)*(-1 + w)*w - u*(-1 + v)*(u - w)*w^2 + (-1 + v)*v*(u - w)*w^2 + u*(u - v)*(-1 + w)*w^2 + (u - v)*v*(-1 + w)*w^2 + v*(-u + v)*(-1 + w)*w^2) / (u^2*(u - v)*(-1 + w)*w - 2*u*(u - v)*v*(-1 + w)*w + (u - v)*v^2*(-1 + w)*w)
            t_prime = (-(u^2*(-1 + v)*v*(u - w)) + 2*u*(-1 + v)*v*(u - w)*w - (-1 + v)*v*(u - w)*w^2)/(u^2*(u - v)*(-1 + w)*w - 2*u*(u - v)*v*(-1 + w)*w + (u - v)*v^2*(-1 + w)*w)
            pairs.add((s_prime, t_prime))

    # print(pairs)
    
    def get(s, t):
        return {
            (s, t),
            (s / t, 1 / t),
            (s, -1 - s - t),
            (s / t, -(1 + s + t) / t),
            (-s / (1 + s + t), -1 / (1 + s + t)),
            (-s / (1 + s + t), -t / (1 + s + t))
        }
    
    pairs = set.union(*(get(s, t) for s, t in pairs))
    return set((x^3 + s*x + t) / (x^2 - x) for s, t in pairs)

def ratlEquivClassesCase3(print_progress=False):
    classes = defaultdict(set)
    total = len(deg3ratls)
    candidates = deepcopy(deg3ratls)
    if print_progress:
        print(f"{total - len(candidates)} out of {total} done", end="\r")
    while candidates:
        f = candidates.pop()
        orbit = getOrbit(f)
        classes[f] = orbit
        candidates -= orbit
        if print_progress:
            print(f"{total - len(candidates)} out of {total} done", end="\r")
    if print_progress:
        print()
    return classes

In [ ]:
getOrbit((x^3 + 3*x + 6) / (x^2 - x))

{(3, 4), (3, 6)}


{(x^3 + 6*x + 2)/(x^2 + 6*x),
 (x^3 + 4*x + 3)/(x^2 + 6*x),
 (x^3 + 3*x + 4)/(x^2 + 6*x),
 (x^3 + 6*x + 5)/(x^2 + 6*x),
 (x^3 + 3*x + 6)/(x^2 + 6*x),
 (x^3 + 4*x + 6)/(x^2 + 6*x)}

In [14]:
classes = ratlEquivClassesCase3(print_progress=True)
print(len(classes))
pprint(classes.keys())

109 out of 109 done
16
dict_keys([(x^3 + 7*x + 4)/(x^2 + 16*x), (x^3 + 14*x + 7)/(x^2 + 16*x), (x^3 + 5*x + 14)/(x^2 + 16*x), (x^3 + x + 13)/(x^2 + 16*x), (x^3 + 9*x + 8)/(x^2 + 16*x), (x^3 + 11*x + 11)/(x^2 + 16*x), (x^3 + 10*x + 3)/(x^2 + 16*x), (x^3 + 10*x + 7)/(x^2 + 16*x), (x^3 + 8*x + 4)/(x^2 + 16*x), (x^3 + 15*x + 6)/(x^2 + 16*x), (x^3 + 1)/(x^2 + 16*x), (x^3 + 6*x + 1)/(x^2 + 16*x), (x^3 + 9*x + 4)/(x^2 + 16*x), (x^3 + 15*x + 2)/(x^2 + 16*x), (x^3 + 7*x + 1)/(x^2 + 16*x), (x^3 + 14*x + 1)/(x^2 + 16*x)])


In [ ]:
s, t = F(2), F(3)
ratlEquiv((x^3 + (-s / (1 + s + t))*x + (-t / (1 + s + t)))/(x^2 - x), (x^3 + s*x + t)/(x^2 - x))

In [97]:
def ratlEquivClassesCase2():
    out = set() 
    if q % 3 == 1: 
        C = [F.primitive_element()^i for i in range(3)]
    else: 
        C = [1]
    for t in C: 
        out.add((x^3 + t)/x)
    
    for t in F: 
        if t != 0: 
            out.add((x^3 + x^2 + t)/x)
    
    return out 

def qnr(): 
    qrs = quadratic_residues(q)
    a = 2
    while a in qrs: 
        a += 1
    return a

def ratlEquivClassesCase1():
    if q % 6 == 1: 
        b = -qnr() 
        a = F(-9)/F(b) 
        return {(x^3 + a*x)/(b*x^2+1)}
    elif q % 6 == 3: 
        a = -qnr() 
        return {x^3, x^3 + a*x}
    elif q % 6 == 4: 
        return "I'll implement this later"
    else: 
        return {x^3}

def ratlEquivClasses():
    case1 = ratlEquivClassesCase1()
    case2 = ratlEquivClassesCase2() 
    case3 = ratlEquivClassesCase3().keys()
    return case1 | case2 | case3
        

In [103]:
def smallRamifications(): 
    sigma = qnr()
    return {x^3, x^3 -3*x, (x^3+3*sigma*x)/(3*x^2+sigma), x^3 - 3 * sigma * x}    

In [104]:
equivs = ratlEquivClasses()
pprint(equivs)
pprint(smallRamifications())

{(2*x^3 + 6*x)/(x^2 + 2),
 (x^3 + 1)/x,
 (x^3 + x^2 + 1)/x,
 (x^3 + 3*x + 1)/(x^2 + 6*x),
 (x^3 + 4*x + 1)/(x^2 + 6*x),
 (x^3 + 6*x + 1)/(x^2 + 6*x),
 (x^3 + 2)/x,
 (x^3 + x^2 + 2)/x,
 (x^3 + 3)/x,
 (x^3 + 3)/(x^2 + 6*x),
 (x^3 + x^2 + 3)/x,
 (x^3 + x^2 + 4)/x,
 (x^3 + x^2 + 5)/x,
 (x^3 + 3*x + 5)/(x^2 + 6*x),
 (x^3 + x^2 + 6)/x,
 (x^3 + 4*x + 6)/(x^2 + 6*x)}
{(5*x^3 + 3*x)/(x^2 + 1), x^3, x^3 + 4*x, x^3 + 5*x}


In [138]:
def ramCase(f): 
    df = factor(derivative(f).numerator())
    deg2termcounter = 0 
    for term in list(df): 
        deg = term[0].degree()
        if deg == 3: 
            return 3
        elif deg == 4: 
            return 5 
        elif deg == 2: 
            deg2termcounter += term[1]
    if deg2termcounter == 1: 
        return 2 
    elif deg2termcounter == 2: 
        return 4 
    else: 
        return 1 
    

def ram4Classes(): 
    cases = {1: 0, 2: 0, 3:0, 4: 0, 5: 0}
    equivs = ratlEquivClasses() 
    for f in equivs: 
        cases[ramCase(f)] += 1 

    for f in smallRamifications(): 
        cases[ramCase(f)] -= 1
    return cases
    

In [147]:
print(ram4Classes())


{1: 21, 2: 63, 3: 44, 4: 60, 5: 64}
